In [ ]:
import psi4
import numpy as np
from scipy import linalg as LA

In [ ]:
# ==> Set default program options <==
psi4.set_memory('1000 MB')
psi4.set_options({'basis':'cc-pvdz','scf_type': 'pk','e_convergence': 1e-8})


In [ ]:
# ==> Geometry and Basis Set <==
mol=psi4.geometry("O\nH 1 1.1\nH 1 1.1 2 104\nsymmetry c1\n")
E_nuc = mol.nuclear_repulsion_energy()
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
ndocc,nbf = wfn.nalpha(), wfn.basisset().nbf()
mints = psi4.core.MintsHelper(wfn.basisset())

In [ ]:
# ==> SCF Iterations <==
SCF_E, E_old = 0.0, 0.0
MAXITER, E_conv = 40,1.0e-6

In [ ]:
# Build and Orthogonormalize Overlap Matrix
S = np.asarray(mints.ao_overlap())
A = LA.sqrtm(np.linalg.inv(S))
S_p = A.dot(S.dot(A))

In [ ]:
# Core Hamiltonian Initial Guess
H=np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())
# Transform The Fock Matrix
F_p=A.dot(H.dot(A)) # First Guess F=H
# Compute Coefficient & Density Matrices
vals,vecs=np.linalg.eigh(F_p)
C=A.dot(vecs)
C_occ=C[:,:ndocc]
D=np.einsum('pi,qi->pq',C_occ,C_occ)

In [ ]:
print('==> Starting SCF Iterations <==\n')
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix (Section XII)
    I=np.asarray(mints.ao_eri())
    J=np.einsum('rs,pqrs->pq',D,I)
    K=np.einsum('rq,pqrs->ps',D,I)
    F=H+2*J-K
    # Compute SCF energy
    SCF_E = E_nuc+np.einsum('pq->',(H+F)*D) #<your formula here>
    print(F'SCF Iteration {scf_iter}: Energy = {SCF_E:.8f} dE = {SCF_E - E_old:.8f}')
    # Check Convergence
    if (abs(SCF_E - E_old) < E_conv): break
    E_old=SCF_E
    # Compute new coefficient & density matrices (Section X & XI) 
    F_p=A.dot(F.dot(A))
    vals,vecs=np.linalg.eigh(F_p)
    C=A.dot(vecs)
    C_occ=C[:, :ndocc]
    D= np.einsum('pi,qi->pq',C_occ,C_occ)
    # Check MAXITER
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")
# Post iterations
print('\nSCF converged.')
print(F'Final RHF Energy: {SCF_E:.6f} [Eh]')